# ST590 Project 3
## Jovanni Catalan & Sergio Mora

In [1]:
import pandas as pd

### Introduction
You should discuss the goals of the notebook, introduce your data set, and give the source for your data set

The goal of this notebook is to have a clear understanding of obesity rates in Mexico, Peru, and Colombia based on multiple metrics collected. This data comes to us from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition+) which gathered this data from [Dataset for estimation of obesity levels based on eating habits and physical condition in individuals from Colombia, Peru and Mexico](https://www.sciencedirect.com/science/article/pii/S2352340919306985?via%3Dihub). 

The data set has 17 columns and 2,111 observations. The columns are:

Gender: Patients gender {object}  
  -  female  
  -  male 

Age: Patients age {float}

Height: Patients height {float}

Weight: Patients weight {float}

family_history_with_overweight: If the patients has a family history of overweight people {object}
  - Yes  
  - No
  
FAVC: Frequent Consumption of High Caloric Food {objects}  
  - Yes
  - No
  
FCVC: Frequent Consumption of Vegetables {float} <font color='red'>This is different from the article need review</font> 
  - 1 through 3 meaning 
    - never  
    - sometimes  
    - always  

NCP: Number of Main Meals (how many meals the patient has daily) {float}  <font color='red'>This is different from the article need review</font> 
  - 1 through 3 meaning 
    - Between 1 and 2  
    - Three   
    - More than three  

CAEC: Consumption of food between meals {object}  
  - No  
  - Sometimes  
  - Frequently  
  - Always  

SMOKE: Does the patient smoke {object}
  - Yes  
  - No  

CH20: Consumption of water in liters {float} <font color='red'>This is different from the article need review</font> 

SCC: Does the patient monitor the calories they consume {object}
  - Yes  
  - No

FAF: How often does the patient have physical activity {float} <font color='red'>This is different from the article need review, There is also some overlap</font> 
  - 0 through 3 meaning 
    - I do not
    - 1 or 2 days
    - 2 or 4 days
    - 4 or 5 days


TUE: How often does the patient utilize technological devices {float} (e.g. phone, video games, TV's, computers, etc.) {float} <font color='red'>This is different from the article need review</font> 
  - 1 through 3 meaning 
    - 0-2 hours
    - 3-5 hours
    - More than 5 hours  

CALC: consumption of alcohol, how often does the patient drink alcohol {object}
  - No (the patient does not drink alcohol)  
  - Sometimes  
  - Frequently  
  - Always  

MTRANS: What type of transportation does the patient normally use {object}
  - Automobile  
  - Motorbike  
  - Bike  
  - Public Transportation  
  - Walking  

NObeyesdad: Patients weight status {object}
  
  - Insufficient Weight  
  - Normal Weight  
  - Obesity Type I  
  - Obesity Type II  
  - Obesity Type III 
  - Overweight Level I
  - Overweight Level II 

In [9]:
Obesity_data = pd.read_csv("ObesityDataSet_raw_and_data_sinthetic.csv")
Obesity_data.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


### Supervised Learning Idea and Data Split

Give a discussion as to why we want to what we are generally trying to do with supervised learning where prediction is our goal. Discuss why we want to split our data into a training and test set.


You should also split the data into a training and test set

### EDA 

You should have a narrative that goes through what you are trying to accomplish in the EDA, why you are looking at a particular graph or statistic, and how you interpret what you’ve made. The EDA should be done on the training data only. You should use pandas-on-spark or spark SQL data frames (but matplotlib is fine)

Part of the final’s purpose is to see if you can judge what should and shouldn’t be included in an EDA.

### Modeling

Next, you should fit three different classes of models (they can be the ones we did in class or you can branch out). You can have a numeric response or a binary response. 

With each model type you use, you should describe the general idea of the model/how it works. These discussion don’t need to be super long, but they should be clear and hit on the most important points about how the model works.

You should use CV to choose among the candidate models for each model type.

  • You should set up a pipeline in pyspark for each of your models  
  
  • At least one of the pipelines should include at least two transformations prior to the model fit (estimator)  
  
  • You can use the same set of transformations for multiple models (if appropriate)